<a href="https://colab.research.google.com/github/smduarte/spbd-2324/blob/main/lab6/SPBD_Labs_spark3_exercise_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Spark SQL Exercises

For this set of exercises, you should use SQL statements, as
much as possible!

Check this online resource for some help with [SQL queries](https://www.codecademy.com/learn/learn-sql/modules/learn-sql-queries/cheatsheet)

In [1]:
#@title Install Pyspark
!pip install --quiet pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
#@title Download "Os Maias"
!wget -q -O os_maias.txt https://www.dropbox.com/s/n24v0z7y79np319/os_maias.txt?dl=0
!wc os_maias.txt

   5877  216896 1292368 os_maias.txt


##1. Sorted Word Frequency

1.1) Create a [Spark SQL](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/index.html) program that counts the number of occurrences of each word in "Os Maias" novel, sorting them by frequency (the words with higher occurrence first).


In [3]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext

try :
  lines = sc.textFile('os_maias.txt') \
  .filter( lambda line : len(line) > 1 ) \
  .map( lambda line : Row( line = line ) )

  linesDF = spark.createDataFrame( lines )
  linesDF.createOrReplaceTempView("OSMAIAS")

  x = spark.sql("SELECT word, count(*) as freq FROM \
                  (SELECT explode(split(line, ' ')) as word FROM OSMAIAS) \
                  GROUP BY word ORDER BY freq DESC")

  x.show(20)
except Exception as err:
  print(err)
  sc.stop()

+----+----+
|word|freq|
+----+----+
|  de|8308|
|   a|6720|
|   o|6602|
| que|4846|
|   e|4441|
|   -|3535|
|  um|3004|
| com|2792|
|  do|2564|
|  da|2200|
| uma|2154|
|  os|1762|
|para|1733|
|   E|1602|
| não|1586|
|  em|1505|
|  no|1439|
|  se|1427|
|  as|1401|
|  ao|1391|
+----+----+
only showing top 20 rows



1.2) Create a Spark Dataframes program that computes the top 10 most used words in "Os Maias" novel.

In [4]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext

try :
  lines = sc.textFile('os_maias.txt') \
  .filter( lambda line : len(line) > 1 ) \
  .map( lambda line : Row( line = line ) )

  linesDF = spark.createDataFrame( lines )
  linesDF.createOrReplaceTempView("OSMAIAS")

  x = spark.sql("SELECT word, count(*) as freq FROM \
                  (SELECT explode(split(line, ' ')) as word FROM OSMAIAS) \
                  GROUP BY word ORDER BY freq DESC LIMIT 10")

  x.show(20)
except Exception as err:
  print(err)
  sc.stop()

+----+----+
|word|freq|
+----+----+
|  de|8308|
|   a|6720|
|   o|6602|
| que|4846|
|   e|4441|
|   -|3535|
|  um|3004|
| com|2792|
|  do|2564|
|  da|2200|
+----+----+



##2. Weblog Analysis

Consider a set of log files captured during a DDOS (*Distributed Denial of Service*) attack, containing information for the web accesses performed during the attack to the server.

The log files contain text lines as shown below, with TAB as the separator:

date |IP_source | status_code | operation | URL | execution time |
-|-|-|-|-|-
timestamp  | string | int | string | string| float |
2016-12-06T08:58:35.318+0000|37.139.9.11|404|GET|/codemove/TTCENCUFMH3C|0.026

In [5]:
#@title Download the dataset
!wget -q -O web.log https://www.dropbox.com/s/0r8902uj9yum7dg/web.log?dl=0
!wc web.log
!head -1 web.log

  143457   860742 11758533 web.log
2016-12-06T08:58:35.318+0000 37.139.9.11 404 GET /codemove/TTCENCUFMH3C 0.026  


2.1. Count the number of unique IP addresses involved in the DDOS attack.


In [9]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]') \
						.appName('weblog').getOrCreate()
sc = spark.sparkContext
try :
    lines = sc.textFile('web.log')
    logRows = lines.filter( lambda line : len(line) > 0 ) \
                   .map( lambda line : line.split(' ') ) \
                   .map( lambda l : Row( date = l[0][0:18], \
				    		            ipSource = l[1], retValue = l[2], \
                            op = l[3], url = l[4], time = float(l[5])))

    logRowsDF = spark.createDataFrame( logRows )
    logRowsDF.createOrReplaceTempView("WEBLOG")

#    x = spark.sql("SELECT count(*) FROM \
#                    (SELECT DISTINCT ipSource FROM WEBLOG)")

    x = spark.sql("SELECT count(DISTINCT ipSource) FROM WEBLOG")

    x.show()
    sc.stop()
except Exception as err:
    print(err)
    sc.stop()

+------------------------+
|count(DISTINCT ipSource)|
+------------------------+
|                     167|
+------------------------+



2.2. For each interval of 10 seconds, provide the following information: [number of requests, average execution time, maximum time, minimum time]

In [10]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]') \
						.appName('weblog').getOrCreate()
sc = spark.sparkContext
try :
    lines = sc.textFile('web.log')
    logRows = lines.filter( lambda line : len(line) > 0 ) \
                   .map( lambda line : line.split(' ') ) \
                   .map( lambda l : Row( date = l[0], \
				    		            ipSource = l[1], retValue = l[2], \
                            op = l[3], url = l[4], time = float(l[5])))

    logRowsDF = spark.createDataFrame( logRows )
    logRowsDF.createOrReplaceTempView("WEBLOG")

    spark.udf.register("toInterval", lambda x : x[0:18])

    x = spark.sql("SELECT toInterval(date) as intervalo, count(*) as requests, \
                      min(time), max(time), mean(time) FROM WEBLOG \
                      GROUP BY intervalo \
                      ORDER BY requests DESC")

    x.show()
    sc.stop()
except Exception as err:
    print(err)
    sc.stop()

+------------------+--------+---------+---------+-------------------+
|         intervalo|requests|min(time)|max(time)|         mean(time)|
+------------------+--------+---------+---------+-------------------+
|2016-12-06T09:00:4|   12135|    0.007|    9.537|0.32576440049443844|
|2016-12-06T09:01:1|   10552|    0.004|    1.506|0.14804018195602886|
|2016-12-06T09:00:1|    9719|    0.225|   34.406|  7.857372672085602|
|2016-12-06T09:01:0|    8747|    0.005|    1.624|0.17546667428832857|
|2016-12-06T09:00:5|    8062|    0.006|    1.905|0.20009092036715637|
|2016-12-06T08:59:3|    8015|    0.056|   67.441| 11.210152214597631|
|2016-12-06T08:59:4|    7947|    0.914|   65.706|  7.761815779539431|
|2016-12-06T08:59:0|    6914|    0.018|   81.659|  38.53438212322824|
|2016-12-06T09:00:0|    6882|    0.017|   45.314|  8.649971519907023|
|2016-12-06T09:00:3|    6771|    0.007|    26.53| 1.6047638458130256|
|2016-12-06T09:00:2|    6616|    0.014|   25.847|  4.611345223700128|
|2016-12-06T08:59:1|

2.3. Create an inverted index that, for each interval of 10 seconds, has a list of (unique) IPs executing accesses (to each URL).

In [11]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]') \
						.appName('weblog').getOrCreate()
sc = spark.sparkContext
try :
    lines = sc.textFile('web.log')
    logRows = lines.filter( lambda line : len(line) > 0 ) \
                   .map( lambda line : line.split(' ') ) \
                   .map( lambda l : Row( date = l[0][0:18], \
				    		            ipSource = l[1], retValue = l[2], \
                            op = l[3], url = l[4], time = float(l[5])))

    logRowsDF = spark.createDataFrame( logRows )
    logRowsDF.createOrReplaceTempView("WEBLOG")

    spark.udf.register("toInterval", lambda x : x[0:18])

    x = spark.sql("SELECT toInterval(date) as intervalo, collect_set( ipSource) as Ips FROM WEBLOG \
                          GROUP BY intervalo \
                          ORDER BY intervalo DESC")

    x.show()
    sc.stop()
except Exception as err:
    print(err)
    sc.stop()

+------------------+--------------------+
|         intervalo|                 Ips|
+------------------+--------------------+
|2016-12-06T10:03:2|[106.37.189.69, 1...|
|2016-12-06T10:03:1|[106.37.189.69, 2...|
|2016-12-06T10:03:0|[106.37.189.69, 1...|
|2016-12-06T10:02:5|[106.37.189.69, 2...|
|2016-12-06T10:02:4|[106.37.189.69, 2...|
|2016-12-06T10:02:3|[106.37.189.69, 2...|
|2016-12-06T10:02:2|[106.37.189.69, 2...|
|2016-12-06T10:02:1|[106.37.189.69, 2...|
|2016-12-06T10:02:0|[106.37.189.69, 1...|
|2016-12-06T10:01:5|[106.37.189.69, 1...|
|2016-12-06T10:01:4|[106.37.189.69, 1...|
|2016-12-06T10:01:3|[106.37.189.69, 2...|
|2016-12-06T10:01:2|[106.37.189.69, 2...|
|2016-12-06T10:01:1|[106.37.189.69, 2...|
|2016-12-06T10:01:0|[106.37.189.69, 2...|
|2016-12-06T09:55:3|    [88.157.128.134]|
|2016-12-06T09:32:2|     [64.74.215.139]|
|2016-12-06T09:27:0|[106.37.189.69, 2...|
|2016-12-06T09:26:5|[106.37.189.69, 2...|
|2016-12-06T09:26:4|[106.37.189.69, 2...|
+------------------+--------------

2.4. Create an inverted index that, for each interval of 15 seconds, has a list of (unique) IPs executing accesses (to each URL).

In [13]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]') \
						.appName('weblog').getOrCreate()
sc = spark.sparkContext
try :
    lines = sc.textFile('web.log')
    logRows = lines.filter( lambda line : len(line) > 0 ) \
                   .map( lambda line : line.split(' ') ) \
                   .map( lambda l : Row( date = l[0], \
				    		            ipSource = l[1], retValue = l[2], \
                            op = l[3], url = l[4], time = float(l[5])))

    logRowsDF = spark.createDataFrame( logRows ).withColumn('date', col('date').cast("timestamp"))
    logRowsDF.createOrReplaceTempView("WEBLOG")

    x = spark.sql("SELECT from_unixtime((unix_timestamp(date) div 15) * 15) as intervalo, collect_set(ipSource) as Ips \
                      FROM WEBLOG GROUP BY intervalo ORDER BY intervalo")

    x.show()
    sc.stop()
except Exception as err:
    print(err)
    sc.stop()

+-------------------+--------------------+
|          intervalo|                 Ips|
+-------------------+--------------------+
|2016-12-06 08:58:30|[202.106.16.36, 2...|
|2016-12-06 08:58:45|[2a01:488:66:1000...|
|2016-12-06 08:59:00|[2a01:488:66:1000...|
|2016-12-06 08:59:15|[2a01:488:66:1000...|
|2016-12-06 08:59:30|[2a01:488:66:1000...|
|2016-12-06 08:59:45|[2a01:488:66:1000...|
|2016-12-06 09:00:00|[2a01:488:66:1000...|
|2016-12-06 09:00:15|[2a01:488:66:1000...|
|2016-12-06 09:00:30|[2a01:488:66:1000...|
|2016-12-06 09:00:45|[2a01:488:66:1000...|
|2016-12-06 09:01:00|[187.60.170.22, 1...|
|2016-12-06 09:01:15|[2001:41d0:8:11c6...|
|2016-12-06 09:01:30|[114.215.150.13, ...|
|2016-12-06 09:01:45|[103.18.4.13, 120...|
|2016-12-06 09:02:00|[120.55.83.30, 18...|
|2016-12-06 09:02:15|[185.15.43.51, 20...|
|2016-12-06 09:02:30|[120.52.73.97, 17...|
|2016-12-06 09:02:45|[177.54.250.18, 1...|
|2016-12-06 09:03:15|      [185.15.42.51]|
|2016-12-06 09:03:30|      [185.15.42.51]|
+----------